Zadanie1: Zastanów się dlaczego funkcja kosztu MSE może nie być najlepszym pomysłem dla tego problemu?

MSE jest stosowany w przypadku odwzorowań liniowych. Przy założeniu, że odwzorowujemy obraz 1:1 jego zastosowanie ma teoretycznie sens. Jednakże, może być obecnych wiele szumów, a przy nim mamy pojedyńczy wektor wyjściowy. To tak jakby przywidywać coś binanego. Dodatkowo kernele gaussowskie mają tendencję do znikającego gradientu.
Lepsza zdaje się np. kernel inverse quadratic dla WAE-MMD, albo błąd rekonstrukcji.

Zadanie2: Jaka jest różnica między funkcją ReLU i LeakyReLU. Jakie to ma konsekwencje?

LReLU jest przekształceniem ReLU, służącej do rozwiązań nieliniowych. Z tą różnicą, iż LReLU nie przekształca wszystkie wartości ujemne jako zero, tylko używa małego parametru nachylenia pozostawiając informacje o ich ujemności. ReLU może okazać się gorszy, gdy część neuronów stanie się nieaktywna przez to że zeruje ujemne wartości. Może też przyczynić się do braku zbieżności 

Zadanie3: Jak zachowuje się funkcja kosztu w kolejnych epokach? Z czego to może wynikać? Czy to ułatwia czy utrudnia dostosowyanie hiperparametrów?

Raz jest większa dla generatora, raz dla dyskryminatora. Nie ułatwia to z całą pewnością dobór hiperparametrów. 
Sytuacja wynika z tego, że uczymy adwersarialnie, tak więc funkcja kosztu dopasuwuje się do każdego kroku.

Zadanie4: Generator może realizować swój cel ucząć się bardzo dobrze generować tylko pewien podzbiór danych (np jedną cyfrę). Zjawisko to nosi nazwę mode collaps. Poszukaj jak można rozwiązać ten problem.

1. Aktualizowanie funkcji strat generatora w celu propagacji wstecznej;
Niewprowadzone GAN- przewidywanie przyszłych dyskryminacji przy pomocy funkcji kosztu dyskryminatora
2. Modyfikowanie dyskryminatora w celu podejmowania decyzji na podstawie wielu próbek tej samej klasy
3. Strata Wassersteina- naprawa odległości między dwoma rozkładami prawdopodobieństwa.



Zadanie5: CIFAR10 to dataset zawierający 60 tysięcy kolorowych obrazków rozmiaru 32 x 32 pikseli należących do 10 różnych klas. Poniżej przykładowe wizualizacje.

Nie wiem czy tego Pan oczekiwał w tym zadaniu, natomiast wykorzystałam implementację z ćwiczeń i innych źródeł. Kod jest podobny do kilku innych, dopasowany do moich danych. Nie widziałam innej opcji, ponieważ tak tworzy się sieć więc implementacja też tak powinna wyglądać (funkcje wymagają tych samych argumentów), nic nowego nie wymyślę :) Przynajmniej tak myślę :)
Główny problem stanowiło coś przy przekształceniu liniowym..kompletnie nie wiedziałam jak go rozwiązać. Optymalizatory i funkcję kosztu wybrałam takie jak na ćwiczeniach.

In [ ]:
#biblioteki
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets, transforms
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor, Normalize, Compose
from torch.utils.data import DataLoader

#używam GPU 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#DataLoader
dataset = CIFAR10(root='.', train=True, transform=ToTensor(), download=True)
loader = DataLoader(dataset, batch_size=15, shuffle=True)
train_dataset= dataset
test_dataset= dataset
batch_size= 100
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


#implementacja sieci
num_classes = 10
class ConvCIFAR(nn.Module):                                             #tu korzystam z implementowanych na ćwiczeniach różnych kodów + z tego źródła:https://towardsdatascience.com/convolutional-neural-networks-explained-9cc5188c4939
    def __init__(self, num_classes=10):
        super(ConvCIFAR, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=48, kernel_size=5, stride=1, padding=2), 
            nn.BatchNorm2d(48),                               
            nn.ReLU(),                                              
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=48, out_channels=96, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(21*21*96, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)  #nie rozumiem czemu tu wyskakuje błąd???
        return out
  
#model
model = ConvCIFAR(num_classes).to(device)

#funckja kosztu i optymalizator
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#Trenujemy model na danych
num_epochs= 5
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        #Krok Forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        #Krok Backward i optymalizacja
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #co 100 przejscie wyswietl obecnna wartosc funkcji kosztu
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

model.eval() 

with torch.no_grad():
    correct = 0 
    total = 0  
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy of the model on the 60000 test images: {100 * correct / total}')

#Zapisujemy nasz model
torch.save(model.state_dict(), 'model.ckpt')

